In [1]:
include("slim_reader.jl")
include("Encoder.jl")
using Knet

┌ Info: Recompiling stale cache file /home/ec2-user/.julia/compiled/v1.2/PyCall/GkzkC.ji for PyCall [438e738f-606a-5dbb-bf0a-cddfbfd45ab0]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file /home/ec2-user/.julia/compiled/v1.2/Knet/f4vSz.ji for Knet [1902f260-5fb4-5aff-8c31-6271790ab950]
└ @ Base loading.jl:1240
┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = top-level scope at slim_reader.jl:6
└ @ Core /home/ec2-user/ilker/spatial-relations/slim_reader.jl:6


In [2]:
BATCH_SIZE = 50
slim_data_dir = "/home/ec2-user/efs/slim-dataset"

"/home/ec2-user/efs/slim-dataset"

In [3]:
syn_train_files = Slim_Reader.get_train_filenames(slim_data_dir, true, false)
# Construct training dataset object for synthetic data
syn_train_data = Slim_Reader.make_dataset_object(syn_train_files, BATCH_SIZE)




Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.



2019-11-19 18:55:39.031976: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2019-11-19 18:55:39.087916: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300075000 Hz
2019-11-19 18:55:39.088380: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7967f40 executing computations on platform Host. Devices:
2019-11-19 18:55:39.088405: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0-3
OMP: Info #156: KMP_AFFINITY: 4 available OS procs
OMP: Info #157: K

Dict{String,Any} with 4 entries:
  "sess"         => PyObject <tensorflow.python.client.session.Session object a…
  "filenames"    => PyObject <tf.Tensor 'Placeholder:0' shape=(?,) dtype=string>
  "iterator"     => PyObject <tensorflow.python.data.ops.iterator_ops.Iterator …
  "next_element" => (((PyObject <tf.Tensor 'IteratorGetNext:0' shape=(?, 9, ?, …

In [4]:
# Iterate through the dataset and get the next batch
syn_train_batch = Slim_Reader.get_next_batch(syn_train_data, KnetArray{Float32}, false, true);

In [5]:
aimages, acaptions, acameras = getdata(syn_train_batch)

(K32(50,10,128,128,3)[0.6039216⋯], ["There is a magenta sphere behind a pink torus. The sphere is behind the torus.  " "There is a magenta sphere behind a pink torus. The sphere is behind the torus.  " … "There is a aqua cone to the right of a pink torus. The cone is in front of the torus. There is a magenta sphere behind a aqua cone. The sphere is behind the cone. There is a magenta sphere to the right of a pink torus.  " "There is a pink torus to the left of a aqua cone. The torus is in front of the cone. There is a magenta sphere to the left of a aqua cone. The sphere is in front of the cone. There is a magenta sphere to the left of a pink torus.  "; "There is a yellow ico to the right of a peach capsule. The ico is behind the capsule. There is a peach capsule to the left of a yellow ico. There is a yellow ico to the left of a yellow ico.  " "There is a yellow ico in front of a peach capsule. The ico is to the right of the capsule. There is a peach capsule to the left of a yellow ic

In [6]:
# create a dictionary of known words
vocabid = Dict()
vocabid = Dict("" => 1) # both padding and whitespace KnetArray(zeros(EMBEDDING_SIZE))
vocab = reshape(KnetArray(zeros(EMBEDDING_SIZE)),(EMBEDDING_SIZE,1))

64×1 KnetArray{Float64,2}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [7]:
function getembed(vocabid, vocab, w)
    if haskey(vocabid, w)
        return vocab[:,vocabid[w]]
    else
        println("There is no such word in the vocab!!!!")
    end
end

getembed (generic function with 1 method)

In [8]:
function updatevocabs!(vocabid, vocab, w)
    if haskey(vocabid, w)
        println("This word already exists...")
        return vocabid, vocab
    else
        if size(vocab,2) == length(vocabid)
            println("Vocab and vocabid are in order!")
            vocabid[w] = length(vocabid)+1
            vocab = hcat(vocab, KnetArray(Knet.xavier(EMBEDDING_SIZE)))
            return vocabid, vocab
        else
            return nothing
        end
    end    
end

updatevocabs! (generic function with 1 method)

In [9]:
function arrange(batchofcaptions, vocabid, vocab)
    int_batchofcaptions = []
    for i in 1:BATCH_SIZE
        int_captions = []
        captions = batchofcaptions[i,:]
        for c in captions
            int_cap = []
            str = split(c, " ")
            for i in 1:MAX_LENGTH
               if i <= length(str)
                    vocabid, vocab = updatevocabs!(vocabid, vocab, str[i])
                    push!(int_cap, getembed(vocabid, vocab, str[i])) #KnetArray(Knet.xavier(EMBEDDING_SIZE))
                else
                    push!(int_cap, getembed(vocabid, vocab, "")) # padding, same embedding with whitespace
                end
            end
            push!(int_captions, hcat(int_cap...))
        end
        push!(int_batchofcaptions, int_captions)
    end
    return vocabid, vocab, int_batchofcaptions
end

arrange (generic function with 1 method)

In [10]:
vocabid, vocab, caption_embeds = arrange(acaptions, vocabid, vocab)

Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
This word already exists...
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
This word already exists...
This word already exists...
This word already exists...
Vocab and vocabid are in order!
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This wor

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
Vocab and vocabid are in order!
Vocab and vocabid are in order!
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word al

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
Vocab and vocabid are in order!
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word alread

...

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

(Dict("pink" => 8,"cone" => 16,"right" => 22,"capsule" => 27,"of" => 14,"purple" => 33,"cylinder" => 37,"right." => 44,"to" => 17,"box." => 34…), K64(64,46)[0.0⋯], Any[Any[K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯]], Any[K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯]], Any[K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0

In [53]:
caption_embeds

50-element Array{Any,1}:
 Any[K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯]]
 Any[K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯]]
 Any[K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.15926533165218454⋯], K64(64,60)[-0.1

In [17]:
function createconvinput(caption_embeds)
    o = []
    for i in 1:BATCH_SIZE
        push!(o, cat(transpose(caption_embeds[i])...,dims=3))
    end
    return o = cat(o..., dims=3)
end

createconvinput (generic function with 1 method)

In [54]:
input = createconvinput(caption_embeds)

60×64×500 KnetArray{Float64,3}:
[:, :, 1] =
 -0.159265    0.0664103   0.173372    0.0835269  …   0.153261    0.0107522 
 -0.160854    0.154568   -0.0950832  -0.146337      -0.0810704   0.070392  
  0.0545866  -0.101282   -0.0165242  -0.162246       0.129092    0.0344382 
 -0.144953    0.0137751  -0.0165163  -0.0393191      0.0306962  -0.0129616 
 -0.0621499  -0.0104057   0.133397    0.0613045     -0.139444   -0.0411653 
  0.1615      0.0961471  -0.152915   -0.162758   …  -0.135771   -0.139259  
  0.0545866  -0.101282   -0.0165242  -0.162246       0.129092    0.0344382 
  0.157193    0.0375481   0.0492212   0.123354       0.0474124  -0.00132803
  0.0245874   0.0350601  -0.170649    0.0454988     -0.050592    0.0575238 
  0.0691677  -0.116033   -0.0647789  -0.14249       -0.0357931   0.0856894 
 -0.0621499  -0.0104057   0.133397    0.0613045  …  -0.139444   -0.0411653 
 -0.160854    0.154568   -0.0950832  -0.146337      -0.0810704   0.070392  
  0.1615      0.0961471  -0.152915   -0.1627

In [14]:
@doc conv4

```
conv4(w, x; kwargs...)
```

Execute convolutions or cross-correlations using filters specified with `w` over tensor `x`.

Currently KnetArray{Float32/64,4/5} and Array{Float32/64,4} are supported as `w` and `x`.  If `w` has dimensions `(W1,W2,...,I,O)` and `x` has dimensions `(X1,X2,...,I,N)`, the result `y` will have dimensions `(Y1,Y2,...,O,N)` where

```
Yi=1+floor((Xi+2*padding[i]-Wi)/stride[i])
```

Here `I` is the number of input channels, `O` is the number of output channels, `N` is the number of instances, and `Wi,Xi,Yi` are spatial dimensions.  `padding` and `stride` are keyword arguments that can be specified as a single number (in which case they apply to all dimensions), or an array/tuple with entries for each spatial dimension.

# Keywords

  * `padding=0`: the number of extra zeros implicitly concatenated at the start and at the end of each dimension.
  * `stride=1`: the number of elements to slide to reach the next filtering window.
  * `dilation=1`: dilation factor for each dimension.
  * `mode=0`: 0 for convolution and 1 for cross-correlation.
  * `alpha=1`: can be used to scale the result.
  * `handle`: handle to a previously created cuDNN context. Defaults to a Knet allocated handle.


In [55]:
filter = reshape(KnetArray(Knet.xavier(30*EMBEDDING_SIZE*32)), (1,30,EMBEDDING_SIZE,32)) # output size = embedding/input size
input = reshape(input, (1, MAX_LENGTH, EMBEDDING_SIZE, BATCH_SIZE*NUM_CAPTIONS_PER_SCENE))

1×60×64×500 KnetArray{Float64,4}:
[:, :, 1, 1] =
 -0.159265  -0.160854  0.0545866  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2, 1] =
 0.0664103  0.154568  -0.101282  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3, 1] =
 0.173372  -0.0950832  -0.0165242  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

...

[:, :, 62, 1] =
 0.057343  0.049667  0.0254458  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 63, 1] =
 0.153261  -0.0810704  0.129092  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 64, 1] =
 0.0107522  0.070392  0.0344382  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 1, 2] =
 -0.159265  -0.160854  0.0545866  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2, 2] =
 0.0664103  0.154568  -0.101282  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3, 2] =
 0.173372  -0.0950832  -0.0165242  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

...

[:, :, 62, 2] =
 0.057343  0.049667  0.0254458  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 63, 2] =
 0.153261  -0.0810704  0.129092  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[

In [22]:
@show y = conv4(filter, input, dilation=2)

y = conv4(filter, input, dilation=2) = K64(1,2,32,500)[-0.00806497540191419⋯]


1×2×32×500 KnetArray{Float64,4}:
[:, :, 1, 1] =
 -0.00806498  0.00548288

[:, :, 2, 1] =
 -0.00599031  0.00181671

[:, :, 3, 1] =
 -0.0201087  -0.00848815

...

[:, :, 30, 1] =
 0.00247267  0.00267592

[:, :, 31, 1] =
 -0.00992407  -0.00756464

[:, :, 32, 1] =
 -0.00238596  0.0134257

[:, :, 1, 2] =
 -0.00806498  0.00548288

[:, :, 2, 2] =
 -0.00599031  0.00181671

[:, :, 3, 2] =
 -0.0201087  -0.00848815

...

[:, :, 30, 2] =
 0.00247267  0.00267592

[:, :, 31, 2] =
 -0.00992407  -0.00756464

[:, :, 32, 2] =
 -0.00238596  0.0134257

[:, :, 1, 3] =
 0.00709758  -0.00411739

[:, :, 2, 3] =
 0.00849149  -0.00520644

[:, :, 3, 3] =
 -0.00970155  0.00271794

...

[:, :, 30, 3] =
 -0.00140958  0.00430004

[:, :, 31, 3] =
 -0.00943493  -0.00249416

[:, :, 32, 3] =
 0.00870388  0.00320455

...

[:, :, 1, 498] =
 -0.000171018  0.00704144

[:, :, 2, 498] =
 0.00305854  -0.00116816

[:, :, 3, 498] =
 -0.00987392  0.00343075

...

[:, :, 30, 498] =
 0.00288069  -0.00572725

[:, :, 31, 498] =
 -0.0

In [56]:
function build_angles(tuplebatch)
    output = []
    for i in 1:BATCH_SIZE
        for j in 1:NUM_CAPTIONS_PER_SCENE
            sin_value = tuplebatch[i,j,3]/sqrt(tuplebatch[i,j,1]^2+tuplebatch[i,j,3]^2)
            cos_value = tuplebatch[i,j,1]/sqrt(tuplebatch[i,j,1]^2+tuplebatch[i,j,3]^2)
            push!(output, KnetArray([sin_value, cos_value])) 
        end
    end
    
    return cat(output..., dims=2)
end

build_angles (generic function with 1 method)

In [24]:
tuples = build_angles(acameras)

2×500 KnetArray{Float32,2}:
 0.480815  0.561012   0.303417  0.911382  …   0.549844  0.532776  0.436445
 0.876822  0.827808  -0.952858  0.411561     -0.835268  0.846256  0.899731

In [25]:
struct Chain
    layers
    Chain(layers...) = new(layers)
end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)

In [26]:
struct Layer0; w; b; end
Layer0(ir::Int, ic::Int, o::Int) = Layer0(param(o,ir),param0(o, ic))
(l::Layer0)(x) = (l.w * x .+ l.b)

In [27]:
mlp1=Chain(Layer0(2, 500, 32)) # MLP1 dimensionality 32

Chain((Layer0(P(KnetArray{Float32,2}(32,2)), P(KnetArray{Float32,2}(32,500))),))

In [28]:
mlp2=Chain(Layer0(96, 500, 128), Layer0(128, 500, 196), Layer0(196, 500, 256)) # MLP2 dimensionality 256

Chain((Layer0(P(KnetArray{Float32,2}(128,96)), P(KnetArray{Float32,2}(128,500))), Layer0(P(KnetArray{Float32,2}(196,128)), P(KnetArray{Float32,2}(196,500))), Layer0(P(KnetArray{Float32,2}(256,196)), P(KnetArray{Float32,2}(256,500)))))

In [29]:
cameras_hat = mlp1(tuples)

32×500 KnetArray{Float32,2}:
  0.141209     0.120197   -0.254649     …   0.127867     0.151874 
  0.0961611    0.112193    0.0606252        0.106548     0.0872911
 -0.28311     -0.285335    0.168428        -0.284826    -0.28091  
  0.0248414    0.0261485  -0.00620209       0.0257056    0.0240571
  0.11341      0.0878199  -0.271738         0.0971016    0.126609 
 -0.129496    -0.118638    0.168641     …  -0.122659    -0.134804 
 -0.09752     -0.073368    0.250225        -0.0821182   -0.110011 
  0.237432     0.232926   -0.190405         0.234781     0.238975 
  0.250711     0.247903   -0.186014         0.249164     0.251303 
  0.0966598    0.081083   -0.183516         0.0867608    0.104595 
  0.140311     0.152081   -0.00119005   …   0.148008     0.133549 
  0.201553     0.201094   -0.135663         0.201464     0.201072 
  0.212874     0.220514   -0.0806153        0.217994     0.208046 
  ⋮                                     ⋱                         
  0.0872473    0.0757246  -0.1460

In [47]:
acaptions

50×10 Array{String,2}:
 "There is a magenta sphere behind a pink torus. The sphere is behind the torus.  "                                                                                                                                                                             …  "There is a pink torus to the left of a aqua cone. The torus is in front of the cone. There is a magenta sphere to the left of a aqua cone. The sphere is in front of the cone. There is a magenta sphere to the left of a pink torus.  "                                     
 "There is a yellow ico to the right of a peach capsule. The ico is behind the capsule. There is a peach capsule to the left of a yellow ico. There is a yellow ico to the left of a yellow ico.  "                                                                "There is a yellow ico in front of a peach capsule. The ico is to the right of the capsule. There is a yellow ico in front of a peach capsule. There is a yellow ico to the left of a yellow

In [30]:
captions_hat = convert(KnetArray{Float32}, reshape(reshape(y, (2, 32, 500)), (64, 500)))
#cameras_hat = reshape(cameras_hat, (1, 32, 500))

64×500 KnetArray{Float32,2}:
 -0.00806498   -0.00806498    0.00709758   …   0.00384648   -0.00367163
  0.00548288    0.00548288   -0.00411739      -0.00571553   -0.0105388 
 -0.00599031   -0.00599031    0.00849149      -0.00189562    0.00295994
  0.00181671    0.00181671   -0.00520644      -0.000393969  -0.0149161 
 -0.0201087    -0.0201087    -0.00970155      -0.00357649   -0.0087517 
 -0.00848815   -0.00848815    0.00271794   …  -0.00437046   -0.0109819 
  0.00464793    0.00464793    0.00551684       0.000689547  -0.0104524 
  0.00793494    0.00793494    0.000725682     -0.00218321    0.00726012
  0.00447094    0.00447094   -0.00628809      -0.00614695   -0.00608132
  0.000627468   0.000627468   0.00674077      -0.0047458    -0.00679087
  0.00270634    0.00270634   -0.00166181   …   0.00361286   -0.0076696 
  0.00154702    0.00154702    0.0030544       -0.00893684   -0.00316367
  0.00980724    0.00980724    0.00503244       0.00798584   -0.00381265
  ⋮                                

In [31]:
h = mlp2(cat(captions_hat, cameras_hat, dims=1))

256×500 KnetArray{Float32,2}:
 -0.0360859    -0.0363497     0.021903     …  -0.0377404   -0.0371063  
  0.00714926    0.00826138    0.0032108        0.00696581   0.00483862 
 -0.00580278   -0.00535769    0.00736915      -0.0074177   -0.00810923 
 -0.0166864    -0.0170929     0.00657093      -0.0149029   -0.0157556  
  0.00139298    0.00291641    0.00987268       0.00549143   0.00221083 
  0.0120484     0.00989566   -0.0217406    …   0.00765667   0.0120083  
  0.0438588     0.0428816    -0.034866         0.0421639    0.0440021  
  0.0236932     0.023608     -0.0148296        0.0238262    0.0243223  
 -0.00797079   -0.00708358    0.0123289       -0.00824704  -0.00918967 
  0.0153545     0.018145      0.0127114        0.0179991    0.0142367  
  0.0232619     0.0226076    -0.0218624    …   0.0218066    0.0218776  
  0.00815883    0.00619958   -0.0195106        0.00584225   0.00924884 
 -0.0111423    -0.01168       0.00336636      -0.0131201   -0.0134806  
  ⋮                               

In [32]:
function aggregate(h) # batch of h s
    r = []
    for i in 1:BATCH_SIZE
        push!(r, sum(h[:,10*(i-1)+1:10*i-1], dims=2)./(NUM_CAPTIONS_PER_SCENE-1)) # 1...9, 11...19, ...
    end
    return hcat(r...)
end

aggregate (generic function with 1 method)

In [33]:
r = aggregate(h)

256×50 KnetArray{Float32,2}:
 -0.0259411    -0.00699544   -0.00295132   …  -0.0221325   -0.00572672 
  0.00719565    0.00710085    0.00617718       0.00780766   0.00922926 
 -0.00338457    0.00166213    0.00275872      -0.00167917   0.00274985 
 -0.0116293    -0.00608692   -0.00399232      -0.0104127   -0.00476638 
  0.00519891    0.00868545    0.00833311       0.00722195   0.0145314  
  0.00327318   -0.00971777   -0.0106265    …  -0.00101745  -0.0178926  
  0.0283089     0.00176568   -0.00294899       0.0219884   -0.00517583 
  0.0165673     0.0048767     0.00150838       0.0135396    0.00237648 
 -0.00324591    0.00319165    0.00469363      -0.00158122   0.00613886 
  0.0175559     0.0186921     0.0165171        0.0195832    0.0269349  
  0.0140714    -0.000441001  -0.00344114   …   0.0107669   -0.00404215 
  0.00149064   -0.00965871   -0.0102525       -0.00200911  -0.0173652  
 -0.00887084   -0.00398281   -0.00348387      -0.008805    -0.00657211 
  ⋮                                

In [69]:
function createimgencinput(images) # images size: 50×10×128×128×3
    out = []
    for i in 1:BATCH_SIZE
        push!(out, permutedims(images[i,1:9,:,:,:], (2,3,4,1)))
    end
    return cat(out..., dims=4)
end

createimgencinput (generic function with 1 method)

In [70]:
aimages

50×10×128×128×3 KnetArray{Float32,5}:
[:, :, 1, 1, 1] =
 0.603922  0.603922  0.603922  0.603922  …  0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922  …  0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922  …  0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.60

In [71]:
imgencoderin = createimgencinput(aimages)

128×128×3×450 KnetArray{Float32,4}:
[:, :, 1, 1] =
 0.603922  0.603922  0.603922  0.603922  …  0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922  …  0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922  …  0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922


In [86]:
imgencoderin2 = pool(imgencoderin, window=4, stride=4)

32×32×3×450 KnetArray{Float32,4}:
[:, :, 1, 1] =
 0.603922  0.603922  0.603922  0.603922  …  0.603922   0.603922   0.603922 
 0.603922  0.603922  0.603922  0.603922     0.603922   0.603922   0.603922 
 0.603922  0.603922  0.603922  0.603922     0.603922   0.603922   0.603922 
 0.603922  0.603922  0.603922  0.603922     0.603922   0.603922   0.603922 
 0.603922  0.603922  0.603922  0.603922     0.603922   0.603922   0.603922 
 0.603922  0.603922  0.603922  0.603922  …  0.603922   0.603922   0.603922 
 0.603922  0.603922  0.603922  0.603922     0.603922   0.603922   0.603922 
 0.607843  0.603922  0.603922  0.607843     0.603922   0.603922   0.603922 
 0.662745  0.701961  0.694118  0.623529     0.607843   0.607843   0.607843 
 0.996078  0.996078  1.0       0.980392     0.631373   0.635294   0.635294 
 1.0       0.996078  1.0       1.0       …  0.686275   0.682353   0.690196 
 1.0       1.0       1.0       1.0          0.686275   0.733333   0.717647 
 1.0       1.0       1.0       1.0     

In [73]:
imgencoderin3 = reshape(imgencoderin2, (1,32,32,3,450))

1×32×32×3×450 KnetArray{Float32,5}:
[:, :, 1, 1, 1] =
 0.603922  0.603922  0.603922  0.603922  …  0.466667  1.0  0.996078  0.996078

[:, :, 2, 1, 1] =
 0.603922  0.603922  0.603922  0.603922  …  0.364706  1.0  0.996078  0.996078

[:, :, 3, 1, 1] =
 0.603922  0.603922  0.603922  0.603922  …  0.360784  0.980392  1.0  1.0  1.0

...

[:, :, 30, 1, 1] =
 0.603922  0.603922  0.603922  0.603922  …  0.737255  0.745098  0.788235

[:, :, 31, 1, 1] =
 0.603922  0.603922  0.603922  0.603922  …  0.705882  0.72549  0.780392

[:, :, 32, 1, 1] =
 0.603922  0.603922  0.603922  0.603922  …  0.678431  0.74902  0.764706

[:, :, 1, 2, 1] =
 0.886275  0.886275  0.886275  0.886275  …  0.662745  0.372549  1.0  1.0  1.0

[:, :, 2, 2, 1] =
 0.886275  0.886275  0.886275  0.886275  …  0.333333  0.364706  1.0  1.0  1.0

[:, :, 3, 2, 1] =
 0.886275  0.886275  0.886275  0.886275  …  0.356863  0.988235  1.0  1.0  1.0

...

[:, :, 30, 2, 1] =
 0.886275  0.886275  0.886275  0.886275  …  0.792157  0.760784  0.788235

[:

In [93]:
filt = reshape(KnetArray{Float32}(Knet.xavier(17*17*3)), (17,17,3,1))

17×17×3×1 KnetArray{Float32,4}:
[:, :, 1, 1] =
 -0.0131524   -0.0260908    -0.0126323   …   0.0333466    0.0253228 
 -0.0294708    0.0116292    -0.00919071     -0.0346915    0.00502788
 -0.0389222    0.035171     -0.0196055      -0.00578877   0.0175679 
 -0.0271084   -0.0247136    -0.00143838      0.0304192   -0.0180003 
 -0.043468     0.0312646    -0.0449307       0.0411287   -0.0080703 
 -0.0347844   -0.0048128     0.0353249   …  -0.00841436   0.0212327 
  0.0221347    0.013946     -0.0115273       0.026924    -0.0108775 
  0.0439953   -0.0134532    -0.00859116      0.00187133   0.0475893 
  0.0450662   -0.0038339    -0.0335081       0.0462885    0.0105341 
  0.0392239   -0.00798396   -0.0400397       0.0453491   -0.00767041
  0.0421612    0.000795165   0.0207178   …  -0.0434405   -0.013482  
 -0.0399032   -0.0279642    -0.0436859      -0.0420255    0.0155545 
 -0.0135422    0.00345223    0.0405727      -0.0180285    0.0146569 
  0.0229003    0.0212767     0.00444256      0.0158637  

In [95]:
yimg = conv4(filt, imgencoderin2) # O, N yerine N, O geliyor

16×16×1×450 KnetArray{Float32,4}:
[:, :, 1, 1] =
 -0.0465182  -0.0028515   -0.0393432   …  -0.162117    -0.186029  
 -0.0279319   0.0335755   -0.053983       -0.165018    -0.155604  
  0.0106976   0.088551     0.0368565      -0.133367    -0.117504  
  0.0909052   0.0974444    0.0719         -0.0874661   -0.071668  
  0.0132452   0.00917107   0.0113751      -0.212723    -0.213628  
  0.090264    0.104612     0.0693001   …  -0.144259    -0.141811  
  0.0693725   0.117723     0.064251       -0.208922    -0.199939  
  0.106694    0.155786     0.192191       -0.0726954   -0.0706431 
  0.108452    0.170402     0.177731       -0.114961    -0.12469   
  0.217922    0.186841     0.137262       -0.12889     -0.184958  
  0.131182    0.089701     0.107451    …  -0.024327    -0.105358  
  0.0724495   0.0933341    0.0894381      -0.0477864   -0.113751  
  0.198373    0.163857    -0.00571405     -0.0575263   -0.106721  
  0.0832054  -0.017872    -0.0769817      -0.00167013  -0.0157878 
 -0.0217323  

In [101]:
yimg2 = permutedims(yimg,[1,2,4,3])

16×16×450×1 KnetArray{Float32,4}:
[:, :, 1, 1] =
 -0.0465182  -0.0028515   -0.0393432   …  -0.162117    -0.186029  
 -0.0279319   0.0335755   -0.053983       -0.165018    -0.155604  
  0.0106976   0.088551     0.0368565      -0.133367    -0.117504  
  0.0909052   0.0974444    0.0719         -0.0874661   -0.071668  
  0.0132452   0.00917107   0.0113751      -0.212723    -0.213628  
  0.090264    0.104612     0.0693001   …  -0.144259    -0.141811  
  0.0693725   0.117723     0.064251       -0.208922    -0.199939  
  0.106694    0.155786     0.192191       -0.0726954   -0.0706431 
  0.108452    0.170402     0.177731       -0.114961    -0.12469   
  0.217922    0.186841     0.137262       -0.12889     -0.184958  
  0.131182    0.089701     0.107451    …  -0.024327    -0.105358  
  0.0724495   0.0933341    0.0894381      -0.0477864   -0.113751  
  0.198373    0.163857    -0.00571405     -0.0575263   -0.106721  
  0.0832054  -0.017872    -0.0769817      -0.00167013  -0.0157878 
 -0.0217323  

In [102]:
yimg3 = yimg2[:,:,:,1]

16×16×450 KnetArray{Float32,3}:
[:, :, 1] =
 -0.0465182  -0.0028515   -0.0393432   …  -0.162117    -0.186029  
 -0.0279319   0.0335755   -0.053983       -0.165018    -0.155604  
  0.0106976   0.088551     0.0368565      -0.133367    -0.117504  
  0.0909052   0.0974444    0.0719         -0.0874661   -0.071668  
  0.0132452   0.00917107   0.0113751      -0.212723    -0.213628  
  0.090264    0.104612     0.0693001   …  -0.144259    -0.141811  
  0.0693725   0.117723     0.064251       -0.208922    -0.199939  
  0.106694    0.155786     0.192191       -0.0726954   -0.0706431 
  0.108452    0.170402     0.177731       -0.114961    -0.12469   
  0.217922    0.186841     0.137262       -0.12889     -0.184958  
  0.131182    0.089701     0.107451    …  -0.024327    -0.105358  
  0.0724495   0.0933341    0.0894381      -0.0477864   -0.113751  
  0.198373    0.163857    -0.00571405     -0.0575263   -0.106721  
  0.0832054  -0.017872    -0.0769817      -0.00167013  -0.0157878 
 -0.0217323  -0.13